# Final Retrieval System

This is a (slightly) improved retrieval system based on the BM25 retrieval.
It implements a custom index, query extension and stemming, sequential dependence and a simple pipeline.

### Step 1: Import Libraries

Imports TIRA, PyTerrier and Spacy and ensures PyTerrier is loaded.

In [ ]:
from tira.third_party_integrations import ensure_pyterrier_is_loaded, persist_and_normalize_run

# this loads and starts pyterrier so that it also works in the TIRA
ensure_pyterrier_is_loaded()

# PyTerrier must be imported after the call to ensure_pyterrier_is_loaded in TIRA.
import pyterrier as pt

import spacy

### Step 2: Setup for Stopwords

Loads English stopwords from Spacy. Saves them in the file "stopwords.txt" and changes the PyTerrier stopwords file to that. 

In [ ]:
nlp = spacy.load("en_core_web_sm")
spacy_stopwords = set(nlp.Defaults.stop_words)


!rm -Rf /tmp/index
file_path = "stopwords.txt"

with open(file_path, 'w+') as file:
    for element in spacy_stopwords:
        file.write(element+ "\n")

pt.set_property('stopwords.filename','./stopwords.txt')

### Step 3: Import Dataset

Imports the ir_dataset from TIRA and prints the first two queries to see if it worked.

In [ ]:
data = pt.get_dataset('irds:ir-lab-sose-2024/ir-acl-anthology-20240504-training')
print('See the first two queries:')
topics = data.get_topics('title')
print(topics.head(2))

### Step 4: Build an Index

Builds a custom iterable index. 
Stems words and removes stopwords previously defined.
Block indexing is enabled.

In [ ]:
print('Build index:')
indexer = pt.IterDictIndexer("/tmp/index", overwrite = True, blocks = True,meta = {'docno':100, 'text': 20480}, stemmer = 'PorterStemmer')
!rm -Rf /tmp/index
index_ref = indexer.index(data.get_corpus_iter())

print('Done. Index is created')

### Step 5: Setup for Retrieval-Pipeline

First definitions of index and retrieval models used.
"sdm" enables use of Sequential Dependance Model.

In [ ]:
index = pt.IndexFactory.of(index_ref)
bm25 = pt.BatchRetrieve(index, wmodel="BM25", k1=1.5, b=0.75, verbose=True)
pl2 = pt.BatchRetrieve(index, wmodel="PL2", c=7.0, verbose=True)
sdm = pt.rewrite.SequentialDependence()

Secondly, definition of the query expansion and the actual pipeline.

In [ ]:
#Query Expansion
bo1 = pt.rewrite.Bo1QueryExpansion(index) 
rm3 = pt.rewrite.RM3(index)

#Pipeline
pipe = (bm25 % 100) >> rm3 >> sdm >> pl2

### Step 6: Create the Run

Creates the run on the queries from the dataset.

In [ ]:
print('Create run')

run = pipe(topics)

print('Done, run was created')

### Step 7: Save the Runfile

In [ ]:
persist_and_normalize_run(run, 'retrieval_system', default_output='../runs')